In [1]:
from selenium import webdriver
import pandas as pd

option = webdriver.ChromeOptions()

browser = webdriver.Chrome(executable_path='/Users/blakeduncan/Documents/chromedriver', options=option)

In [3]:
def thesportsdb_login(username, password):
    browser.get("https://www.thesportsdb.com/")
    browser.find_element_by_xpath("//*[@id='header']/nav/div/div[2]/ul/li[6]/a[1]").click()
    browser.find_element_by_xpath("//*[@id='feature']/div/div[3]/div/form/div[1]/input").send_keys(username)
    browser.find_element_by_xpath("//*[@id='feature']/div/div[3]/div/form/div[2]/input").send_keys(password)
    browser.find_element_by_xpath("//*[@id='rememberme']").click()
    browser.find_element_by_xpath("//*[@id='feature']/div/div[3]/div/form/div[4]/input").click()

def add_event(league, date, starttime, season, hometeam, awayteam, week):
    for i in range(len(league)):
        browser.get(f"https://www.thesportsdb.com/edit_event_add.php?l={league[i]}")
        browser.find_element_by_xpath("//*[@id='datepicker']").send_keys(date[i])
        browser.find_element_by_xpath("//*[@id='starttime']").send_keys(starttime[i])
        browser.find_element_by_xpath("//*[@id='season']").send_keys(season[i])
        browser.find_element_by_xpath("//*[@id='hometeam']").send_keys(hometeam[i])
        browser.find_element_by_xpath("//*[@id='awayteam']").send_keys(awayteam[i])
        browser.find_element_by_xpath("//*[@id='round']").send_keys(week[i])
        browser.find_element_by_xpath("//*[@id='submit']").click()

def add_score(league, date, starttime, season, hometeam, awayteam, homescore, awayscore, week):
    for i in range(len(league)):
        browser.get(f"https://www.thesportsdb.com/edit_event_add.php?l={league[i]}")
        browser.find_element_by_xpath("//*[@id='datepicker']").send_keys(date[i])
        browser.find_element_by_xpath("//*[@id='starttime']").send_keys(starttime[i])
        browser.find_element_by_xpath("//*[@id='season']").send_keys(season[i])
        browser.find_element_by_xpath("//*[@id='hometeam']").send_keys(hometeam[i])
        browser.find_element_by_xpath("//*[@id='awayteam']").send_keys(awayteam[i])
        browser.find_element_by_xpath("//*[@id='homescore']").send_keys(homescore[i])
        browser.find_element_by_xpath("//*[@id='awayscore']").send_keys(awayscore[i])
        browser.find_element_by_xpath("//*[@id='round']").send_keys(week[i])
        browser.find_element_by_xpath("//*[@id='submit']").click()

def add_player(team, fullname, dob, position, nationality):
    for i in range(len(team)):
        browser.get(f"https://www.thesportsdb.com/edit_player_add.php?t={team[i]}")
        browser.find_element_by_xpath("//*[@id='fullname']").send_keys(fullname[i])
        browser.find_element_by_xpath("//*[@id='datepicker']").send_keys(dob[i])
        browser.find_element_by_xpath("//*[@id='position']").send_keys(position[i])
        browser.find_element_by_xpath("//*[@id='countries']").send_keys(nationality[i])
        browser.find_element_by_xpath("//*[@id='submit']").click()

In [ ]:
thesportsdb_login("cardinaldiehard25", "HHtW3b64iW4SAR@")

In [5]:
event_csv = pd.read_csv("/Users/blakeduncan/Downloads/event.csv")

league = event_csv.league.tolist()
date = event_csv.date.tolist()
starttime = event_csv.starttime.tolist()
season = event_csv.season.tolist()
hometeam = event_csv.hometeam.tolist()
awayteam = event_csv.awayteam.tolist()
week = event_csv.week.tolist()

add_event(league, date, starttime, season, hometeam, awayteam, week)

In [7]:
score_csv = pd.read_csv("/Users/blakeduncan/Downloads/score.csv")

league = score_csv.league.tolist()
date = score_csv.date.tolist()
starttime = score_csv.starttime.tolist()
season = score_csv.season.tolist()
hometeam = score_csv.hometeam.tolist()
awayteam = score_csv.awayteam.tolist()
homescore = score_csv.homescore.tolist()
awayscore = score_csv.awayscore.tolist()
week = score_csv.week.tolist()

add_score(league, date, starttime, season, hometeam, awayteam, homescore, awayscore, week)

In [ ]:
player_csv = pd.read_csv("/Users/blakeduncan/Downloads/player.csv")

team = player_csv.team.tolist()
fullname = player_csv.fullname.tolist()
dob = player_csv.dob.tolist()
position = player_csv.position.tolist()
nationality = player_csv.nationality.tolist()

add_player(team, fullname, dob, position, nationality)